In [ ]:
from neuro_evolution_body import main
import matplotlib.pyplot as plt
import mujoco
import numpy as np
from ariel.body_phenotypes.robogen_lite.prebuilt_robots.gecko import gecko
from ariel.simulation.environments.simple_flat_world import SimpleFlatWorld



import sys, os, contextlib

@contextlib.contextmanager
def suppress_all_output_c():
    """Suppress everything printed to stdout and stderr, including C-level warnings."""
    devnull = os.open(os.devnull, os.O_WRONLY)
    # Save original stdout/stderr
    old_stdout = os.dup(1)
    old_stderr = os.dup(2)
    # Redirect to /dev/null
    os.dup2(devnull, 1)
    os.dup2(devnull, 2)
    try:
        yield
    finally:
        # Restore original stdout/stderr
        os.dup2(old_stdout, 1)
        os.dup2(old_stderr, 2)
        os.close(devnull)
        os.close(old_stdout)
        os.close(old_stderr)



with suppress_all_output_c():
    x, a = None, None
    if __name__ == "__main__":
        x, a = main(n_population=4, generations=10, p_crossover=0.5, m_rate=0.1)
        plt.figure(figsize=(10, 6))
        progress = [-f for f in a] # Negate the fitness values to get progress
        plt.plot(progress, 'm-', label='Progress Towards Target')
        plt.ylabel("Progress")
        plt.ylim(bottom = 0)
        plt.legend()
        plt.grid(True)

In [1]:
from ariel.simulation.environments.olympic_arena import OlympicArena
from neuro_evolution_controller import main as evolve_robot_controller
from ariel.body_phenotypes.robogen_lite.prebuilt_robots.gecko import gecko


def evaluate_single_robot():
    print(f"Processing robot  ...")
    
    robot_fitness = evolve_robot_controller(
        robot_core_func=gecko,
        world_func=OlympicArena,
        spawn_pos=[-0.8, -0, 0.1],
        time=15,
        population=100,
        p_crossover=0.5,
        m_rate=0.1,
        generations=100,
        hidden_size=8, 
        graph=True
    )
    
    print(f"Finished robot with fitness : {robot_fitness[-1]}")
    return robot_fitness

x, y = evaluate_single_robot()

Processing robot  ...
Processed genome 1/100. Fitness: 5.854188221096835
Processed genome 2/100. Fitness: 5.789485096967905
Processed genome 3/100. Fitness: 6.021633028543097
Processed genome 4/100. Fitness: 5.790018340165836
Processed genome 5/100. Fitness: 5.713080601990516
Processed genome 6/100. Fitness: 5.776757418044117
Processed genome 7/100. Fitness: 5.890661624720538
Processed genome 8/100. Fitness: 5.906768609915824
Processed genome 9/100. Fitness: 5.926653851531207
Processed genome 10/100. Fitness: 5.729876364120882
Processed genome 11/100. Fitness: 5.800328241492248
Processed genome 12/100. Fitness: 5.929978129368853
Processed genome 13/100. Fitness: 5.806770410769779
Processed genome 14/100. Fitness: 5.910983238929961
Processed genome 15/100. Fitness: 5.856962148196752
Processed genome 16/100. Fitness: 5.826523057674526
Processed genome 17/100. Fitness: 5.792334956106488
Processed genome 18/100. Fitness: 5.833037896945888
Processed genome 19/100. Fitness: 5.894333535840088

Process ForkPoolWorker-412:


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/s4ddo/.local/share/uv/python/cpython-3.12.9-linux-x86_64-gnu/lib/python3.12/multiprocessin │
│ g/pool.py:856 in next                                                                            │
│                                                                                                  │
│   853 │   def next(self, timeout=None):                                                          │
│   854 │   │   with self._cond:                                                                   │
│   855 │   │   │   try:                                                                           │
│ ❱ 856 │   │   │   │   item = self._items.popleft()                                               │
│   857 │   │   │   except IndexError:                                                             │
│   858 │   │   │   │   if self._index == self._length:                                            │
│   859 │   │   │   │   │   self._pool = None                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:25                                                                                   │
│                                                                                                  │
│   22 │   print(f"Finished robot with fitness : {robot_fitness[-1]}")                             │
│   23 │   return robot_fitness                                                                    │
│   24                                                                                             │
│ ❱ 25 x, y = evaluate_single_robot()                                                              │
│   26                                                                                             │
│                                                                                                  │
│ in evaluate_single_robot:9                                                                       │
│                                                                                                  │
│    6 def evaluate_single_robot():                                                                │
│    7 │   print(f"Processing robot  ...")                                                         │
│    8 │                                                                                           │
│ ❱  9 │   robot_fitness = evolve_robot_controller(                                                │
│   10 │   │   robot_core_func=gecko,                                                              │
│   11 │   │   world_func=OlympicArena,                                                            │
│   12 │   │   spawn_pos=[-0.8, -0, 0.1],                                                          │
│                                                                                                  │
│ /home/s4ddo/Documents/Masters_Year_1/P1/EvoAI/neuro_evolution_controller.py:258 in main          │
│                                                                                                  │
│   255 │   │   offsprings                     = crossover(parents, p_crossover)                   │
│   256 │   │                                                                                      │
│   257 │   │   offsprings                     = mutate(offsprings, m_rate)                        │
│ ❱ 258 │   │   f_offspring                    = evaluate(offsprings, robot_core_func, world_fun   │
│   259 │   │   population, population_fitness = survivior_selection(                              │
│   260 │   │   │   parents, parents_f, offsprings, f_of

In [ ]:
# import random
# if __name__ == "__main__":
#     runs = 2
#     generations = 30
#     all_best = []

#     for r in range(runs):
#         # (optioneel) reproduceerbaar maken
#         seed = 1234 + r
#         np.random.seed(seed)
#         random.seed(seed)

#         _, f_best = main(time=30, population=10, generations=generations)
#         all_best.append(np.asarray(f_best, dtype=float))

#     # Lengtes gelijk trekken (voor de zekerheid)
#     min_len = min(len(b) for b in all_best)
#     A = np.vstack([b[:min_len] for b in all_best])   # shape: (runs, min_len)

#     mean = A.mean(axis=0)
#     std  = A.std(axis=0, ddof=1)                     # sample std
#     gens = np.arange(min_len)                        # of np.arange(1, min_len+1)

#     plt.figure(figsize=(10, 6))
#     plt.plot(gens, mean, 'm-', label='Mean best per generation')
#     plt.fill_between(gens, mean - std, mean + std, alpha=0.2, label='±1 SD')
#     plt.xlabel('Generation')
#     plt.ylabel('Distance to goal (lower is better)')
#     plt.ylim(bottom=0)
#     plt.legend()
#     plt.grid(True)
#     plt.tight_layout()
#     plt.show()

In [ ]:
from pathlib import Path
from networkx import DiGraph
from networkx.readwrite import json_graph
import json

def load_graph_from_json(
    load_file: Path | str,EvoAI
) -> DiGraph:
    with Path(load_file).open("r", encoding="utf-8") as f:
        data = json.load(f)

    graph = json_graph.node_link_graph(data, edges="edges", directed=True)
    return graph

In [2]:
from ariel.body_phenotypes.robogen_lite.constructor import (
    construct_mjspec_from_graph,
)
from neuro_evolution_body import DATA
from neuro_evolution_controller import nn_control
import mujoco
import numpy as np
from mujoco import viewer


mujoco.set_mjcb_control(None) # DO NOT REMOVE
world = OlympicArena()
gecko_core= gecko()

world.spawn(gecko_core.spec, [-0.8, -0, 0.1])
model = world.spec.compile()
data = mujoco.MjData(model) # type: ignore
geoms = world.spec.worldbody.find_all(mujoco.mjtObj.mjOBJ_GEOM)
to_track = [data.bind(geom) for geom in geoms if "core" in geom.name]

input_size  = len(data.qpos)  # e.g., 6 positions + 6 velocities
hidden_size = 8
output_size = model.nu
history = []

mujoco.set_mjcb_control(lambda m,d: nn_control(m, d, to_track, np.array(x), history, input_size, hidden_size, output_size))

viewer.launch(
    model=model,  # type: ignore
    data=data,
)

/home/s4ddo/Documents/Masters_Year_1/P1/EvoAI/.venv/lib/python3.12/site-packages/glfw/__init__.py:917: GLFWError: (65548) b'Wayland: The platform does not provide the window position'
  warnings.warn(message, GLFWError)
